In [1]:
#%matplotlib notebook
from z_utils.Tracker import Tracker
from z_utils.init_detector_05 import *
from DiffuseMap import *
import socket
import json

In [2]:
class ROS_Linker:
    def __init__(self):
        self.CaptureObject=None
        self.list_params=None
        self.video_out=None
        self.Tracker=None
        self.diffuse_map = None

    def initParams(self):
        print('initParam...')

        path_in = 'video_frames/video_15/' #Папка в которой лежат либо фотографии либо видео
        file_name = ''   #Для папки с фоторгафиями оставляем пустой. Для видео пишем ТОЛЬКО название видео.
        video_name = file_name + '.mp4'
        path_out = './output/'
        clear_dir_00(path_out) #Очищаем всю папку path_out
        path_out_for_DM_images = path_out + path_in + '/'  # куда складывать картинки с диффузной картой

        #Сохранение для папки с кадрами.
        if len(file_name) == 0:
            name = path_in
            ext = '.mp4'
            self.video_in = path_in + file_name + '/'
            self.video_out = path_out + name + '/' + name[:-1] + ext
        #Сохранение для видео.
        else:
            name = path_in
            ext = '.mp4'
            self.video_in = path_in + video_name
            self.video_out = path_out + name + '/' + file_name + ext
            

        if not os.path.exists(path_out_for_DM_images):
            os.makedirs(path_out_for_DM_images)

        clear_dir_00(path_out_for_DM_images) # Очищаем папку куда сложили дифф карту
        clear_dir_00('save_fig/')

        print('video_in', self.video_in)
        print('video_out', self.video_out)
        print('path_out_for_DM_images', path_out_for_DM_images)


        input_type_flag = 3  # Тип входных данных:0 - видео, 1 - папка с файлами, 2 - rtsp-поток (тут веб-камера сейчас), 3 - ftp-поток (не реализовано), 4 - ROS
        flag_cut_and_save_objects = 0  # вырезаем объекты в ббоксах
        flag_save_tr_mat = 0  # собираем ли траектории в мат
        flag_debug = 0  # надо ли сохранять картинки нарезанные. Не используется
        ########################################################
        p_opt_flow = 0.99  # 0- идем по предикт, 1- идем по оптикал флоу
        ########### Настраиваем тут предиктор ##################
        predictor_type = 4  # 0 - yolo, 1 - detectron2 (cadet), 2 - detectron2 (pedet), 3 - face_detection, 4 - facenet_pytorch
        ########################################################
        video_adapter = 1 # Выбор видеоадаптера: 0 - CPU, 1 - GPU
        ########################################################
        add_holistic = 0 # 0 - выключить скелет, 1 - включить скелет
        ########################################################
        show_3Dpoints_holistic = 0 # 1 - Будет рисоваться скелет с фрагментами в папке save_fig 
        ########################################################
        show3D_one_points = 0 # 1 - Будет рисоваться только 1 точка головы с фрагментами в папке save_fig 
        ########################################################
        show_BBOX = 0 # Флаг для отрисовки ББокса и превью.
        ########################################################
        show_diffusion = 0 # 0 - выключена дифф мапа, 1 - включена на запись
        ########################################################

        thr_confidence = 0.99 #Отвечает за удаления ложных айдишников

        #Используем предиктор 4 - фейснет пайторч
        main_predictor = None
        if predictor_type == 0:
            main_predictor = detector_cocos(thr_confidence)
        elif predictor_type == 1:
            main_predictor = detector_detectron_cadet(model='cadet')
        elif predictor_type == 2:
            main_predictor = detector_detectron_cadet(model='pedet')
        elif predictor_type == 3:
            main_predictor = detector_detectron_cadet()
        elif predictor_type == 4:
            main_predictor = detector_facenet_pytorch(thr_confidence, video_adapter)
        else:
            pass
        
        #Для потока в вебкамеры, заменяем self.video_in на 0. Сохранять фото
        if input_type_flag == 2:
            self.video_in = 0
            self.video_out = path_out + path_in + 'rtsp_video' + ext
        if input_type_flag == 3:
            self.video_in = "udp://@172.16.119.221:1111"
            self.video_out = path_out + path_in + 'rtsp_video' + ext
        
            
        dscr_TL_00 = descr_745_init(11)  # 11 - фейснет
        ####################################

        self.list_params = {
            'path_out': path_out,
            'dscr_TL_00': dscr_TL_00, #Выбор дескриптора. Используем 11 - Facenet, получаеим вектора
            'detector_01': main_predictor,  # детектор для распознавалки объектов
            'detector_02': main_predictor,  # predictor_010,  # ела для распознавалки объектов
            'double_detect_flag': 0,  # если 1, то в фоновом режиме без трека распознаем объекты предиктором 2
            'input_type_flag': input_type_flag, # Выбор модели загрузки файлов/видео/вебкамера
            'predictor_type': predictor_type, #Используем предиктор фейснет_пайторч
            'diffuse_map': self.diffuse_map, 
            'flag_cut_and_save_objects': flag_cut_and_save_objects,
            'flag_save_tr_mat': flag_save_tr_mat,
            'DM_save': None,
            'flag_debug': flag_debug,
            'add_holistic': add_holistic, #Включить скелет или нет
            'path_out_for_DM_images': path_out_for_DM_images,
            'show_diffusion': show_diffusion, #Флаг для дифф карты
            'show_3Dpoints_holistic': show_3Dpoints_holistic, #флаг для 3Д точек
            'show3D_one_points': show3D_one_points, #Флаг для 3Д одной точки
            'show_BBOX': show_BBOX,
            'video_adapter': video_adapter, #Флаг выбора CPU/GPU
            'quality_width': 1920, #Флаг выбора качества
            'resize_coef': 16/9, #Формат изображения
            'compensaition_index' : 8,
            'cannals_hist': 0,  ### 0- гистограмма hue для идентификации объекта, 2- гистограмма яркости

            'flag_classify_traj': 3,  ### классификатор траекторий: 0 - CNN, 1 - lstm, 2- mll, 3 - simple classify
            'path_to_w_classify_traj': 'classify_trajs\\InpOut001_mll_weights_25eps.h5',
            # None, # путь к весам классификатора траекторий
            'flag_prediction': 0,  ### lstm обучена не на том=>  флаг=0 #предсказатель траектории
            'path_to_prediction': None,  # 'q00_person.h5'# путь к весам предсказателя траекторий #

            'thr_color_features': 0.3, #Порог отвечает за сравнение косинусных расстояний. Сравнивает расстояния больше порога
            'check_period': 5,  #Отвечает за количество списков с векторами # forget_horizont_vert #30 - хорошо работает
            'debug_visu': 1,
            'border_edge': 5,  # относительно каких границ ищем новые объекты, used in add_new_object
            'r_edge': 1,  # относительно   границ ищем новые объекты  1- внутри области, -1 снаружи, used in add_new_object

            'l_forget_history': 15,
            'confidence_trashhold': 0.999, #отвечает за то, чтобы добавлялись в список векторизаций только хорошие детекции

            'manual_thr_radius': 2000, # %%%%% Радиус для поиска лица %%%%%

            'threshold': [0.6, 0.7, 0.7],  # three steps's threshold
            'factor': 0.709,  # scale factor 0.709
            'margin': 44, #НЕ ИСПОЛЬЗУЕТСЯ для фейснет_пайторч
            'thr_measure_hue_optflow2predict': None,
            'thr_add_new': 200, # Радиус. Используется в add_new_object
            'thr_correct_opt_flow': p_opt_flow * 350,
            ### если OF скакнуло от предикта больше чем thr_correct_opt_flow, верим предикту
            'prefix_ind_save_0': 10500000,
            'p_opt_flow': p_opt_flow,  # 0- идем по предикт, 1- идем по оптикал флоу
            'start_ind': 1,
            'class_to_interest': np.array(range(10)), #Не очень понял
            'debug_count': -15  # 25*60*5  # 5 минут
        }
        # self.video_out = path_out + filename_out
        #На шаффл История - 10. Чек период - 20
        #На сортированный. История - 10. Чек период - 10

    def init_diffuse_map(self):
        self.diffuse_map = DiffuseMap('pickles/facenet_pytorch.pickle', 0.8, True, 'pickles/DM_vector512.pickle')


    def initCaptureObject(self):
        print('initCaptureObj...')
        self.CaptureObject = Caption_Class_01(self.video_in, self.list_params['input_type_flag'], 1920) #Качество для вебки

    def initTracker(self):
        print('initTracker...')
        self.Tracker= Tracker(self.CaptureObject,self.video_out,self.list_params)
        self.Tracker.InitTracker()

    def TrackImage(self,image=None):
        if self.list_params['input_type_flag'] == 4:
            self.UpdateImage(image)

        return self.Tracker.TrackImage()
    
    def ShutdownTracker(self): #Функция отвечает за сохранение видео
        
        return self.Tracker.ShutdownTracker()
    
    def UpdateImage(self,image):
        self.CaptureObject.updateImageROS(image)
       

    
linker=ROS_Linker()#инициализируем класс
#Сначала инициализируем параметры, связанных с путями до файлов. Затем часть кода для сохранения файлов, если берем видео, или берем папку с файлами
#Далее идет часть кода, где происходит выбор флагов для определенных компонентов кода. Выбираем предиктор, используем фейснет_пайторч. 
#Для потока с вебкамеры заменяем self.video_in. Далее происходит выбор дескриптора, для получения векторизации
#Далее загружаем много параметров, которые используются в коде. Описание есть у всех. Функции далее описаны ниже.
linker.init_diffuse_map()#инициализируем дифф. мапу
linker.initParams()#загружаем все параметры
linker.initCaptureObject()#Загружаем путь к входному видео или в зависимости от флага, переключаемся на камеру
cap_2 = None
if linker.list_params['input_type_flag'] == 4:#Берем входное видео из ROSA
    cap_2 = cv2.VideoCapture(1)
    ret, image = cap_2.read()
    linker.UpdateImage(image)
linker.initTracker()

HOST, PORT = "172.16.119.227", 11309
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

while linker.CaptureObject.isOpened():
    if linker.list_params['input_type_flag'] == 4:
        ret, image = cap_2.read()
        track_result = linker.TrackImage(image)
    else:
        track_result = linker.TrackImage()#Запуск трекера
        result  = '{"objects":['
        is_first = True
        try:
            for i in track_result:
                if i['owned'][-1] > 0.5 and i['age'] > 3:

                    if not is_first:
                        result += ', '
                    else:
                        is_first = False
                    result += '{{ "id": "{}", "x": {}, "y": {}, "z": {}, "name": "{}"}}'.format(i['id'], round(i['3D_points'][0], 4), round(i['3D_points'][1], 4), round(i['3D_points'][2], 4), 'human')

                    # result[i['id']] = {'x': round(i['3D_points'][0], 4), 'y': round(i['3D_points'][1], 4), 'z': round(i['3D_points'][2], 4)}
                elif i['owned'][-1] < 0.5 and i['age'] > 3:

                    if not is_first:
                        result += ', '
                    else:
                        is_first = False
                    result += '{{ "id": "{}", "x": {}, "y": {}, "z": {}, "name": "{}"}}'.format(i['id'], round(i['3DV_lost'][0], 4), round(i['3DV_lost'][1], 4), round(i['3DV_lost'][2], 4), 'human')

                    # result[i['id']] = {'x': round(i['3DV_lost'][0], 4), 'y': round(i['3DV_lost'][1], 4), 'z': round(i['3DV_lost'][2], 4)}

        except:
            pass
            # data = json.dumps(result)
        
        
        result += ']}'
        #data = json.dumps(result)
        data = result
        sock.sendto(bytes(data + "\n", "utf-8"), (HOST, PORT))
        #sock.sendto(bytes(data, "utf-8"), (HOST, PORT))
        print("Sent:{}".format(data))       
        #Что мы получаем в track_result:
        #age - Сколько фрагментов этот человек есть в кадре. age_lost - Сколько кадров фрагмента нет в кадре
        #id - Номер задетекченного человека. #history - Записывает центр Ббокса в каждом кадре объекта.
        #history_X - тоже самое. owned - записывает 1 - если объект есть в кадре, 0 - если пропал из кадра.
        #Используется для обновление истории. Если много нулей, то пропадает из детекта человек.
        #holistic - Записывает 3д точки скелета объекта. descriptor_ - Записывает 512 значений, используется для детекта лица.
        #descriptor_ - векторизация каждого id в каждом кадре объекта. w - ширина Ббокса, всегда одно значение у каждого id.
        #h - высота Ббокса, всегда одно значение у каждого id. class_trajectory - Используется для классификации траектории.
        #radius - Радиус поиска id, если id потерялся из кадра. N14 -используется для проверки. Не важное значение
        #init_posit - центр Ббокса в текущем кадре.



    if cv2.waitKey(10) == 27:  #
        break
        
sock.close()   
#linker.ShutdownTracker() #Сохранение видео
cv2.destroyAllWindows() 

C:\python\platform\Platform_w_one_points3D
initParam...
video_in video_frames/video_15//
video_out ./output/video_frames/video_15//video_frames/video_15.mp4
path_out_for_DM_images ./output/video_frames/video_15//
Running on device: cuda:0
initCaptureObj...
initTracker...
start_ind    1
True
[INFO] Facenet_pytorch took 1.575696 seconds
no init_000: 0
[INFO] Facenet_pytorch took 0.180068 seconds
1920 1080
[INFO] Facenet_pytorch took 0.171150 seconds
Sent:{"objects":[]}
[INFO] Facenet_pytorch took 0.176590 seconds
Sent:{"objects":[]}
[INFO] Facenet_pytorch took 0.185999 seconds
Sent:{"objects":[]}
[INFO] Facenet_pytorch took 0.175999 seconds
Sent:{"objects":[{ "id": "0", "x": 0.111, "y": -0.2916, "z": 1.3622, "name": "human"}]}
[INFO] Facenet_pytorch took 0.190999 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1108, "y": -0.2915, "z": 1.3625, "name": "human"}]}
[INFO] Facenet_pytorch took 0.174115 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1107, "y": -0.2915, "z": 1.3627, "name": "human"}

Sent:{"objects":[{ "id": "0", "x": 0.1109, "y": -0.2918, "z": 1.3647, "name": "human"}]}
[INFO] Facenet_pytorch took 0.183148 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1109, "y": -0.2918, "z": 1.3646, "name": "human"}]}
[INFO] Facenet_pytorch took 0.184908 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1109, "y": -0.2918, "z": 1.3645, "name": "human"}]}
[INFO] Facenet_pytorch took 0.180170 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1109, "y": -0.2917, "z": 1.3645, "name": "human"}]}
[INFO] Facenet_pytorch took 0.186001 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1111, "y": -0.2916, "z": 1.3641, "name": "human"}]}
[INFO] Facenet_pytorch took 0.166596 seconds
Sent:{"objects":[{ "id": "0", "x": 0.111, "y": -0.2915, "z": 1.3639, "name": "human"}]}
[INFO] Facenet_pytorch took 0.173769 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1109, "y": -0.2915, "z": 1.3638, "name": "human"}]}
[INFO] Facenet_pytorch took 0.162023 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1109, "y": -0.2915, "z": 1

[INFO] Facenet_pytorch took 0.180159 seconds
Sent:{"objects":[{ "id": "0", "x": -0.0387, "y": -0.3331, "z": 1.3518, "name": "human"}]}
[INFO] Facenet_pytorch took 0.183384 seconds
Sent:{"objects":[{ "id": "0", "x": -0.0433, "y": -0.3346, "z": 1.3514, "name": "human"}]}
[INFO] Facenet_pytorch took 0.198773 seconds
Sent:{"objects":[{ "id": "0", "x": -0.0929, "y": -0.3492, "z": 1.3465, "name": "human"}]}
[INFO] Facenet_pytorch took 0.180328 seconds
Sent:{"objects":[{ "id": "0", "x": -0.0929, "y": -0.3495, "z": 1.3467, "name": "human"}]}
[INFO] Facenet_pytorch took 0.185633 seconds
Sent:{"objects":[{ "id": "0", "x": -0.0929, "y": -0.3496, "z": 1.3468, "name": "human"}]}
[INFO] Facenet_pytorch took 0.175866 seconds
Sent:{"objects":[{ "id": "0", "x": -0.0928, "y": -0.3495, "z": 1.3468, "name": "human"}]}
[INFO] Facenet_pytorch took 0.187295 seconds
Sent:{"objects":[{ "id": "0", "x": -0.0928, "y": -0.3496, "z": 1.3469, "name": "human"}]}
[INFO] Facenet_pytorch took 0.175515 seconds
Sent:{"obj

[INFO] Facenet_pytorch took 0.176828 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1055, "y": -0.3094, "z": 1.3637, "name": "human"}]}
[INFO] Facenet_pytorch took 0.167328 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1054, "y": -0.3093, "z": 1.3637, "name": "human"}]}
[INFO] Facenet_pytorch took 0.177415 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1055, "y": -0.3093, "z": 1.3638, "name": "human"}]}
[INFO] Facenet_pytorch took 0.176718 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1055, "y": -0.3093, "z": 1.3637, "name": "human"}]}
[INFO] Facenet_pytorch took 0.160080 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1055, "y": -0.3092, "z": 1.3637, "name": "human"}]}
[INFO] Facenet_pytorch took 0.175077 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1055, "y": -0.3088, "z": 1.3636, "name": "human"}]}
[INFO] Facenet_pytorch took 0.157033 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1055, "y": -0.3089, "z": 1.3636, "name": "human"}]}
[INFO] Facenet_pytorch took 0.157495 seconds
Sent:{"objects":[

Sent:{"objects":[{ "id": "0", "x": 0.1057, "y": -0.3095, "z": 1.3642, "name": "human"}]}
[INFO] Facenet_pytorch took 0.156252 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1057, "y": -0.3095, "z": 1.3642, "name": "human"}]}
[INFO] Facenet_pytorch took 0.160244 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1057, "y": -0.3094, "z": 1.3642, "name": "human"}]}
[INFO] Facenet_pytorch took 0.189210 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1057, "y": -0.3093, "z": 1.3643, "name": "human"}]}
[INFO] Facenet_pytorch took 0.175611 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1057, "y": -0.3093, "z": 1.3649, "name": "human"}]}
[INFO] Facenet_pytorch took 0.171877 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1056, "y": -0.3093, "z": 1.365, "name": "human"}]}
[INFO] Facenet_pytorch took 0.159480 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1057, "y": -0.3094, "z": 1.365, "name": "human"}]}
[INFO] Facenet_pytorch took 0.159184 seconds
Sent:{"objects":[{ "id": "0", "x": 0.1057, "y": -0.3094, "z": 1.

...

Sent:{"objects":[{ "id": "0", "x": -0.0025, "y": -0.3089, "z": 1.3681, "name": "human"}]}
[INFO] Facenet_pytorch took 0.160671 seconds
Sent:{"objects":[{ "id": "0", "x": -0.0024, "y": -0.3089, "z": 1.3681, "name": "human"}]}
[INFO] Facenet_pytorch took 0.175056 seconds
Sent:{"objects":[{ "id": "0", "x": -0.0024, "y": -0.3089, "z": 1.3681, "name": "human"}]}
[INFO] Facenet_pytorch took 0.159832 seconds
Sent:{"objects":[{ "id": "0", "x": -0.0022, "y": -0.3088, "z": 1.3678, "name": "human"}]}
[INFO] Facenet_pytorch took 0.156477 seconds
Sent:{"objects":[{ "id": "0", "x": -0.0022, "y": -0.3087, "z": 1.3679, "name": "human"}]}
[INFO] Facenet_pytorch took 0.178960 seconds
Sent:{"objects":[{ "id": "0", "x": -0.0022, "y": -0.3088, "z": 1.368, "name": "human"}]}
[INFO] Facenet_pytorch took 0.171874 seconds
Sent:{"objects":[{ "id": "0", "x": -0.0022, "y": -0.3088, "z": 1.3681, "name": "human"}]}
[INFO] Facenet_pytorch took 0.167078 seconds
Sent:{"objects":[{ "id": "0", "x": -0.0022, "y": -0.3089

[INFO] Facenet_pytorch took 0.179778 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0054, "y": -0.3021, "z": 1.3667, "name": "human"}]}
[INFO] Facenet_pytorch took 0.205961 seconds
Sent:{"objects":[{ "id": "0", "x": 0.009, "y": -0.3003, "z": 1.3652, "name": "human"}]}
[INFO] Facenet_pytorch took 0.189630 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0085, "y": -0.3004, "z": 1.3646, "name": "human"}]}
[INFO] Facenet_pytorch took 0.254245 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0083, "y": -0.3001, "z": 1.364, "name": "human"}]}
[INFO] Facenet_pytorch took 0.208048 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0081, "y": -0.2999, "z": 1.3632, "name": "human"}]}
[INFO] Facenet_pytorch took 0.189695 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0077, "y": -0.3023, "z": 1.3622, "name": "human"}]}
[INFO] Facenet_pytorch took 0.193245 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0047, "y": -0.3122, "z": 1.3566, "name": "human"}]}
[INFO] Facenet_pytorch took 0.184230 seconds
Sent:{"objects":[{ 

Sent:{"objects":[{ "id": "0", "x": 0.0205, "y": -0.3373, "z": 1.3587, "name": "human"}]}
[INFO] Facenet_pytorch took 0.166044 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0205, "y": -0.3374, "z": 1.3586, "name": "human"}]}
[INFO] Facenet_pytorch took 0.171766 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0205, "y": -0.3374, "z": 1.3586, "name": "human"}]}
[INFO] Facenet_pytorch took 0.162792 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0206, "y": -0.3373, "z": 1.3586, "name": "human"}]}
[INFO] Facenet_pytorch took 0.158150 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0208, "y": -0.338, "z": 1.3588, "name": "human"}]}
[INFO] Facenet_pytorch took 0.168013 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0208, "y": -0.338, "z": 1.3587, "name": "human"}]}
[INFO] Facenet_pytorch took 0.165843 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0208, "y": -0.338, "z": 1.3587, "name": "human"}]}
[INFO] Facenet_pytorch took 0.168235 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0209, "y": -0.338, "z": 1.35

Sent:{"objects":[{ "id": "0", "x": 0.0211, "y": -0.3374, "z": 1.3549, "name": "human"}]}
[INFO] Facenet_pytorch took 0.164996 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0211, "y": -0.3374, "z": 1.3551, "name": "human"}]}
[INFO] Facenet_pytorch took 0.170355 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0212, "y": -0.3372, "z": 1.3553, "name": "human"}]}
[INFO] Facenet_pytorch took 0.169765 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0212, "y": -0.3373, "z": 1.3555, "name": "human"}]}
[INFO] Facenet_pytorch took 0.169988 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0213, "y": -0.3374, "z": 1.3556, "name": "human"}]}
[INFO] Facenet_pytorch took 0.159998 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0213, "y": -0.3374, "z": 1.3557, "name": "human"}]}
[INFO] Facenet_pytorch took 0.164437 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0213, "y": -0.3374, "z": 1.3557, "name": "human"}]}
[INFO] Facenet_pytorch took 0.174049 seconds
Sent:{"objects":[{ "id": "0", "x": 0.0217, "y": -0.3376, "z": 

KeyboardInterrupt: 